# import packages

In [1]:
import pandas as pd
import dask.dataframe as dd
import numpy as np
import json
from time import time

from collections import Counter

from konlpy.corpus import kolaw
from konlpy.tag import Hannanum
from konlpy.utils import concordance, pprint
import matplotlib.pyplot as plt
from matplotlib import pyplot

from konlpy.tag import *
from gensim.models.word2vec import Word2Vec

km = Komoran()
tw = Twitter()
han = Hannanum()
kkma = Kkma()

def draw_zipf(count_list, filename, color='blue', marker='o'):
    sorted_list = sorted(count_list, reverse=True)
    plt.plot(sorted_list, color=color, marker=marker)
    plt.xscale('log')
    plt.yscale('log')
    plt.show()
    #pyplot.savefig(filename)


C:\Anaconda2\envs\py36\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


# 나무위키

In [2]:
with open('C:/Users/boogi/Downloads/namuwiki_20180326.json') as f:
    data = json.load(f)

In [3]:
data[0]

{'contributors': ['namubot', 'R:hoon12560'],
 'namespace': '0',
 'text': '#redirect 느낌표\n',
 'title': '!'}

In [4]:
'''
import dask.bag as db
js = db.read_text('C:/Users/boogi/Downloads/namuwiki_1.json').map(json.loads)
js.take(1)
del js
'''

"\nimport dask.bag as db\njs = db.read_text('C:/Users/boogi/Downloads/namuwiki_1.json').map(json.loads)\njs.take(1)\ndel js\n"

In [5]:
len(data)

565985

In [6]:
keys = ['카지노', '호텔', '도박', '강원랜드']
minimal = ''
for i in range(len(data)):
    txt = '%s' % data[i]
    if any([True if key in txt else False for key in keys]):
        for item in txt.split('\\n'):
            if any([True if key in item else False for key in keys]):
                for jtem in item.split('.'):
                    if any([True if key in jtem else False for key in keys]):
                        minimal += jtem + '. '

In [7]:
len(minimal.split('.'))

43000

In [8]:
del data

In [9]:
minimal = minimal.replace(']', '').replace('[', '').replace('*', '').replace('|', '').replace("\\'", "").replace('~', '')

# RNN test

In [10]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Dropout
from keras.optimizers import *
from keras.utils import np_utils
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer

from nltk.tokenize import sent_tokenize
#from konlpy.corpus import kolaw
#from konlpy.tag import Twitter

Using TensorFlow backend.


In [11]:
doc0 = [" ".join(["".join(w) for w, t in tw.pos(s) 
                  if t not in ['Number', "Foreign"]]) for s in sent_tokenize(minimal)]

In [12]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(doc0)
doc = [l for l in tokenizer.texts_to_sequences(doc0) if len(l) > 1]

maxlen = max([len(x) - 1 for x in doc])
vocab_size = len(tokenizer.word_index) + 1

In [13]:
maxlen, vocab_size

(576, 55765)

In [15]:
def generate_data(X, maxlen, vocab_size):
    for sentence in X: 
        inputs = []
        targets = []
        for i in range(1, len(sentence)):
            inputs.append(sentence[0:i])
            targets.append(sentence[i])
        y = np_utils.to_categorical(targets, vocab_size)
        inputs_sequence = sequence.pad_sequences(inputs, maxlen=maxlen)
        yield (inputs_sequence, y)

In [16]:
X = []
Y = []
for x, y in generate_data(doc, maxlen, vocab_size):
    X.append(x)
    Y.append(y)

X = np.concatenate(X)
Y = np.concatenate(Y)

MemoryError: 

In [ ]:
# 국문
def korean_most_common(c, n, pos=None):
    bg = bigram
    if pos is None:
        return bigram[tokenize(c)[0]].most_common(n)
    else:
        return bigram["/".join([c, pos])].most_common(n)
    
def korean_generate_sentence(seed=None, debug=False):
    if seed is not None:
        np.random.seed(seed)
    c = "SS"
    sentence = []
    while True:
        if c not in bigram:
            break
        words, probs = zip(*[(k, v) for k, v in bigram[c].items()])
        idx = np.argmax(np.random.multinomial(1, probs, (1,)))
        w = words[idx]
        
        if w == "SE":
            break            
        
        w2 = w.split("/")[0]
        pos = w.split("/")[1]
        
        if c == "SS":
            sentence.append(w2.title())
        elif c in ["`", "\"", "'", "("]:
            sentence.append(w2)
        elif w2 in ["'", ".", ",", ")", ":", ";", "?"]:
            sentence.append(w2)
        elif pos in ["Josa", "Punctuation", "Suffix"]:
            sentence.append(w2)
        elif w in ["임/Noun", "것/Noun", "는걸/Noun", "릴때/Noun",
                   "되다/Verb", "이다/Verb", "하다/Verb", "이다/Adjective"]:
            sentence.append(w2)
        else:
            sentence.append(" " + w2)
        c = w
        
        if debug:
            print(w)
            
    return "".join(sentence)